In [13]:
import pandas as pd
import re
from datetime import datetime as dt
import pickle

In [12]:
exits=pd.read_csv("../output/cb_exits.csv")
funds=pd.read_csv("../output/cb_funds.csv")

In [14]:
exits.shape

(14759, 45)

In [15]:
funds.shape

(27520, 13)

In [22]:
funds.sort_values("Organization Name")

Transaction Name  \
26701               Seed Round - \t Rhythmm   
25725             Series A -  CryptoKitties   
4338              Series A -  CryptoKitties   
1876                      Seed Round -  DOZ   
876                       Seed Round -  DOZ   
2876                      Seed Round -  DOZ   
4337              Corporate Round -  Harbor   
2046                Series A -  Justmop.com   
46                  Series A -  Justmop.com   
1046                Series A -  Justmop.com   
12117  Venture Round -  Mahindra Logistics    
15583                  Series B -  NimbleRx   
26606                  Series B -  NimbleRx   
20423                  Series A -  NimbleRx   
20415                  Series B -  NimbleRx   
15585                  Series A -  NimbleRx   
26763          Seed Round -  Sheerly Genius   
1050                Seed Round -  SpotDraft   
50                  Seed Round -  SpotDraft   
2050                Seed Round -  SpotDraft   
16148             Debt Financing -  Syntrio   
4470                   Series A -  Tenfold    
24320                Seed Round -  Tenfold    
24321                Seed Round -  Tenfold    
23764                       Series A -  onX   
6272                        Series A -  onX   
12657              Series A - !hey software   
5505          Venture Round - 1-800-DENTIST   
26912                 Seed Round - 10 By 10   
10416              Seed Round - 10% Happier   
...                                     ...   
1694              Venture Round - videogram   
944                  Seed Round - videogram   
460                  Seed Round - videogram   
1612                   Series A - videogram   
2612                   Series A - videogram   
2460                 Seed Round - videogram   
694               Venture Round - videogram   
21248                    Series B - vip.com   
20949                    Series A - vip.com   
21107               Venture Round - vip.com   
441                      Seed Round - vvall   
2441                     Seed Round - vvall   
1441                     Seed Round - vvall   
3222                Venture Round - webtide   
13078            Convertible Note - wedgies   
10905            Convertible Note - wedgies   
10827                  Seed Round - wedgies   
13000                  Seed Round - wedgies   
24539                Seed Round - weeSpring   
15480                 Series B - what3words   
26656              Seed Round - wifi.com.ng   
9011                Seed Round - wireLawyer   
14633                    Series A - wooptix   
27279                Seed Round - writewith   
21660                       Series B - x.ai   
27410                   Seed Round - xendit   
26024              Venture Round - yet2.com   
26983                Seed Round - zPREDICTA   
25106                     Series B - zulily   
4828                      Series D - zulily   

                                    Transaction Name URL  \
26701  https://www.crunchbase.com/funding_round/rhyth...   
25725  https://www.crunchbase.com/funding_round/crypt...   
4338   https://www.crunchbase.com/funding_round/crypt...   
1876   https://www.crunchbase.com/funding_round/capse...   
876    https://www.crunchbase.com/funding_round/capse...   
2876   https://www.crunchbase.com/funding_round/capse...   
4337   https://www.crunchbase.com/funding_round/harbo...   
2046   https://www.crunchbase.com/funding_round/justm...   
46     https://www.crunchbase.com/funding_round/justm...   
1046   https://www.crunchbase.com/funding_round/justm...   
12117  https://www.crunchbase.com/funding_round/mahin...   
15583  https://www.crunchbase.com/funding_round/nimbl...   
26606  https://www.crunchbase.com/funding_round/nimbl...   
20423  https://www.crunchbase.com/funding_round/nimbl...   
20415  https://www.crunchbase.com/funding_round/nimbl...   
15585  https://www.crunchbase.com/funding_round/nimbl...   
26763  https://www.crunchbase.com/funding_round/sheer...   
1050   https://www.crunchbase.co

<h2>Remove duplicate investments by multiple VCs</h2> 

In [23]:
f_uq = funds[funds.duplicated(subset=["Funding Type", "Organization Name"], keep = "last") == False]

In [24]:
f_uq.shape

(18597, 13)

In [25]:
f_uq["Funding Type"].value_counts()

Seed                        4405
Series A                    3856
Series B                    3222
Venture - Series Unknown    2021
Series C                    1974
Series D                     975
Convertible Note             514
Debt Financing               449
Series E                     389
Private Equity               273
Angel                        174
Series F                     144
Funding Round                 58
Post-IPO Equity               36
Series G                      31
Post-IPO Debt                 24
Grant                         12
Corporate Round               10
Series H                       8
Secondary Market               8
Initial Coin Offering          4
Non-equity Assistance          4
Equity Crowdfunding            3
Product Crowdfunding           2
Post-IPO Secondary             1
Name: Funding Type, dtype: int64

In [30]:
f_uq.groupby(["Organization Name", "Funding Type"])["Money Raised"].apply(sum)

Organization Name     Funding Type            
\t Rhythmm            Seed                              $120,000
 CryptoKitties        Series A                       $12,000,000
 DOZ                  Seed                            $1,500,000
 Harbor               Corporate Round                $28,000,000
 Justmop.com          Series A                               NaN
 Mahindra Logistics   Venture - Series Unknown    ₹2,472,000,000
 NimbleRx             Series A                       $12,000,000
                      Series B                       $28,000,000
 Sheerly Genius       Seed                              $120,000
 SpotDraft            Seed                              $550,000
 Syntrio              Debt Financing                    $400,000
 Tenfold              Seed                            $1,350,000
                      Series A                       $15,800,000
 onX                  Series A                       $20,300,000
!hey software         Series A             

<h3> Use only Seed --> Series C data </h3>

In [31]:
f_rounds = f_uq[(f_uq["Funding Type"] == "Seed") |
                (f_uq["Funding Type"] == "Series A") |
                (f_uq["Funding Type"] == "Series B") |
                (f_uq["Funding Type"] == "Series C")]

In [32]:
f_rounds["Funding Type"].value_counts()

Seed        4405
Series A    3856
Series B    3222
Series C    1974
Name: Funding Type, dtype: int64

In [33]:
f_date = f_rounds.pivot(index = "Organization Name" , columns = "Funding Type", values = "Announced Date")

# df.rename(columns=lambda x: x.replace('$', ''), inplace=True)

f_date.rename(columns = {"Seed": "Seed_date",
               "Series A": "A_date",
               "Series B": "B_date",
               "Series C": "C_date"}, inplace = True)

In [34]:
f_money = f_rounds.pivot(index = "Organization Name", columns = "Funding Type", values = "Money Raised")

In [35]:
f_trans = pd.concat([f_money, f_date], axis = 1, join = 'outer', ignore_index = False)
f_trans

Funding Type                 Seed     Series A     Series B     Series C  \
Organization Name                                                          
\t Rhythmm               $120,000         None         None         None   
 CryptoKitties               None  $12,000,000         None         None   
 DOZ                   $1,500,000         None         None         None   
 Justmop.com                 None          NaN         None         None   
 NimbleRx                    None  $12,000,000  $28,000,000         None   
 Sheerly Genius          $120,000         None         None         None   
 SpotDraft               $550,000         None         None         None   
 Tenfold               $1,350,000  $15,800,000         None         None   
 onX                         None  $20,300,000         None         None   
!hey software                None  $10,000,000         None         None   
10 By 10                 $120,000         None         None         None   
10% Happier            $1,400,000         None         None         None   
1000 Markets             $500,000         None         None         None   
1000memories              $15,000   $2,520,000         None         None   
1006.tv                      None         None  $10,000,000         None   
100Plus                  $750,000         None         None         None   
101 Edu                       NaN         None         None         None   
10X Genomics                 None         None  $55,000,000  $55,000,000   
11 Honore                    None   $8,000,000         None         None   
117go                        None          NaN         None         None   
123Feng.Com                  None  $13,000,000         None         None   
15Five                 $2,200,000         None         None         None   
17hats                 $1,250,000   $4,000,000         None         None   
19pay                        None         None   $6,230,000         None   
1DocWay                       NaN         None         None         None   
1Huddle                  $125,000         None         None         None   
1mg                          None   $6,000,000  $16,000,000  $15,000,000   
1stdibs                      None         None  $42,000,000         None   
20n                    $2,100,000         None         None         None   
20x200                       None     $800,000         None         None   
...                           ...          ...          ...          ...   
tomoguides                    NaN         None         None         None   
tracx                        None         None   $3,500,000  $18,000,000   
tray.io                $2,200,000   $5,000,000         None         None   
true[x]                      None   $4,200,000   $8,000,000  $11,650,000   
turntable.fm                 None   $7,000,000         None         None   
tuul                   $2,400,000         None         None         None   
twoXAR                 $3,400,000  $10,000,000         None         None   
uBeam                    $750,000  $10,000,000  $20,000,000         None   
uBiome                       None   $4,500,000  $22,000,000         None   
uCiC ( you see I See)    $500,000         None         None         None   
uVore                         NaN         None         None         None   
unGlue                 $1,500,000         None         None         None   
unspun, inc.                  NaN         None         None         None   
vArmour                      None         None  $15,000,000         None   
vLinx                        None         None  $10,000,000         None   
vidIQ                  $2,000,000         None         None         None   
videogram                $500,000   $4,099,999         None         None   
vip.com                      None   $9,190,000  $41,240,000         None   
vvall                         NaN         None         None         None   
wedgies                  $700,000         None  

In [36]:
f_trans["index"] = f_trans.index
print(f_trans.shape)
print(f_trans["index"].nunique())

(9480, 9)
9480


## Run some checks

In [37]:
f_trans[f_trans.duplicated(keep = False)].sort_values("index")

Empty DataFrame
Columns: [Seed, Series A, Series B, Series C, Seed_date, A_date, B_date, C_date, index]
Index: []

In [38]:
f_trans[f_trans.index == "Agami System"]

Funding Type       Seed Series A Series B     Series C Seed_date A_date  \
Organization Name                                                         
Agami System       None     None     None  $50,000,000      None   None   

Funding Type      B_date        C_date         index  
Organization Name                                     
Agami System        None  Nov 21, 2007  Agami System

In [20]:
# col = f_trans.columns.values

# for col in f_trans:
#     print(f_trans[col].notnull().value_counts())

In [57]:
print(f_trans.shape)
f_trans.dropna(axis = 0, how = "any", thresh = 5).shape

(9480, 9)


(2691, 9)

In [58]:
f_trans

Funding Type                 Seed     Series A     Series B     Series C  \
Organization Name                                                          
\t Rhythmm               $120,000         None         None         None   
 CryptoKitties               None  $12,000,000         None         None   
 DOZ                   $1,500,000         None         None         None   
 Justmop.com                 None          NaN         None         None   
 NimbleRx                    None  $12,000,000  $28,000,000         None   
 Sheerly Genius          $120,000         None         None         None   
 SpotDraft               $550,000         None         None         None   
 Tenfold               $1,350,000  $15,800,000         None         None   
 onX                         None  $20,300,000         None         None   
!hey software                None  $10,000,000         None         None   
10 By 10                 $120,000         None         None         None   
10% Happier            $1,400,000         None         None         None   
1000 Markets             $500,000         None         None         None   
1000memories              $15,000   $2,520,000         None         None   
1006.tv                      None         None  $10,000,000         None   
100Plus                  $750,000         None         None         None   
101 Edu                       NaN         None         None         None   
10X Genomics                 None         None  $55,000,000  $55,000,000   
11 Honore                    None   $8,000,000         None         None   
117go                        None          NaN         None         None   
123Feng.Com                  None  $13,000,000         None         None   
15Five                 $2,200,000         None         None         None   
17hats                 $1,250,000   $4,000,000         None         None   
19pay                        None         None   $6,230,000         None   
1DocWay                       NaN         None         None         None   
1Huddle                  $125,000         None         None         None   
1mg                          None   $6,000,000  $16,000,000  $15,000,000   
1stdibs                      None         None  $42,000,000         None   
20n                    $2,100,000         None         None         None   
20x200                       None     $800,000         None         None   
...                           ...          ...          ...          ...   
tomoguides                    NaN         None         None         None   
tracx                        None         None   $3,500,000  $18,000,000   
tray.io                $2,200,000   $5,000,000         None         None   
true[x]                      None   $4,200,000   $8,000,000  $11,650,000   
turntable.fm                 None   $7,000,000         None         None   
tuul                   $2,400,000         None         None         None   
twoXAR                 $3,400,000  $10,000,000         None         None   
uBeam                    $750,000  $10,000,000  $20,000,000         None   
uBiome                       None   $4,500,000  $22,000,000         None   
uCiC ( you see I See)    $500,000         None         None         None   
uVore                         NaN         None         None         None   
unGlue                 $1,500,000         None         None         None   
unspun, inc.                  NaN         None         None         None   
vArmour                      None         None  $15,000,000         None   
vLinx                        None         None  $10,000,000         None   
vidIQ                  $2,000,000         None         None         None   
videogram                $500,000   $4,099,999         None         None   
vip.com                      None   $9,190,000  $41,240,000         None   
vvall                         NaN         None         None         None   
wedgies                  $700,000         None  

# Select Columns to be from exits.csv to merge with funds.csv

In [61]:
exits["Categories"]

0        Content, News, Social Bookmarking, Social Medi...
1               Credit, Finance, FinTech, Personal Finance
2        Graphic Design, Photo Editing, Web Apps, Web D...
3          B2B, B2C, Big Data, Bitcoin, Financial Services
4        Catering, Food Delivery, Logistics, Office Adm...
5        Apps, E-Commerce, Home Services, Location Base...
6                 Enterprise Software, Internet, SMS, VoIP
7        Angel Investment, Finance, Funding Platform, I...
8        Analytics, Business Information Systems, Email...
9        Analytics, Big Data, Business Intelligence, Cr...
10       Beauty, Communities, Digital Media, Fashion, L...
11                     Marketplace, Retail, Sporting Goods
12                          E-Commerce, Fashion, Lifestyle
13       Apps, Dating, Mobile Apps, Social Media, Socia...
14       Big Data, Cloud Computing, Enterprise Software...
15                      E-Learning, Education, Marketplace
16                                       FinTech, Softwa

In [62]:
print(exits.columns)
e_reduce = exits[["Organization Name", "Categories", "Category Groups", "Founded Date", 
                  "Number of Exits", "Number of Founders", "Number of Employees", "Number of Funding Rounds",
                  "Funding Status", "Last Funding Date", "Last Funding Type", "Last Funding Amount", "Last Equity Funding Amount",
                  "Number of Investors", "Acquisition Status", "IPO Status", "IPO Date", "Money Raised at IPO", "Valuation at IPO",
                  "Number of Articles", "Operating Status", "Total Funding Amount", "Total Equity Funding Amount"]]

e_reduce

Index(['Organization Name', 'Organization Name URL', 'Category Groups',
       'Headquarters Location', 'Description', 'CB Rank (Organization)',
       'Headquarters Regions', 'Operating Status', 'Founded Date',
       'Closed Date', 'Company Type', 'Website', 'LinkedIn', 'Facebook',
       'Twitter', 'Investor Type', 'Investment Stage',
       'Number of Portfolio Organizations', 'Number of Investments',
       'Number of Lead Investments', 'Number of Exits', 'Categories',
       'Number of Founders', 'Number of Employees', 'Number of Funding Rounds',
       'Funding Status', 'Last Funding Date', 'Last Funding Type',
       'Last Funding Amount', 'Last Equity Funding Amount',
       'Last Equity Funding Type', 'Total Equity Funding Amount',
       'Total Funding Amount', 'Number of Lead Investors',
       'Number of Investors', 'Acquisition Status', 'IPO Status', 'IPO Date',
       'Delisted Date', 'Money Raised at IPO', 'Valuation at IPO',
       'Contact Email', 'Phone Number', 'Num

Organization Name  \
0                            Reddit   
1                      Credit Karma   
2                             Canva   
3              Libra Credit Network   
4                             Zesty   
5                        TaskRabbit   
6                            Twilio   
7                         AngelList   
8                          SendGrid   
9                            Womply   
10                            POPxo   
11                     SidelineSwap   
12                     The RealReal   
13                     EastMeetEast   
14                          Algolia   
15                            Udemy   
16                             Qwil   
17                           App.io   
18                             Eaze   
19                         PlanGrid   
20                           Averon   
21                        Carousell   
22                          Life360   
23                        Prenetics   
24                           Pipefy   
25                         Iterable   
26                        ContaAzul   
27                          Tradesy   
28                           uBiome   
29                          BarkBox   
...                             ...   
14729                       FGL.com   
14730                        Caarly   
14731                   engageSPARK   
14732                   Ring Router   
14733                 Alpha Outpost   
14734                     NetworkBe   
14735                    Leaseville   
14736                         Exist   
14737                        Stacck   
14738                      Daatrics   
14739                  Make It Work   
14740                     Squabbler   
14741                      DFR Asia   
14742                    XCast Labs   
14743                    MaestroDev   
14744                      Doxcheck   
14745                       miiddle   
14746                       Markkit   
14747                 MoAnima, Inc.   
14748       Mobile Content Networks   
14749                       webtide   
14750              Media Matchmaker   
14751                 Organic To Go   
14752           Information Gateway   
14753           Praized Media, Inc.   
14754                  Maria Health   
14755                      GrowSari   
14756                      Clupedia   
14757  Spark Marketing and Research   
14758      Advent Integra Solutions   

                                              Categories  \
0      Content, News, Social Bookmarking, Social Medi...   
1             Credit, Finance, FinTech, Personal Finance   
2      Graphic Design, Photo Editing, Web Apps, Web D...   
3        B2B, B2C, Big Data, Bitcoin, Financial Services   
4      Catering, Food Delivery, Logistics, Office Adm...   
5      Apps, E-Commerce, Home Services, Location Base...   
6               Enterprise Software, Internet, SMS, VoIP   
7      Angel Investment, Finance, Funding Platform, I...   
8      Analytics, Business Information Systems, Email...   
9      Analytics, Big Data, Business Intelligence, Cr...   
10     Beauty, Communities, Digital Media, Fashion, L...   
11                   Marketplace, Retail, Sporting Goods   
12                        E-Commerce, Fashion, Lifestyle   
13     Apps, Dating, Mobile Apps, Social Media, Socia...   
14     Big Data, Cloud Computing, Enterprise Software...   
15                    E-Learning, Education, Marketplace   
16                                     FinTech, Software   
17     Android, Customer Service, Enterprise Software...   
18     E-Commerce, Health Care, Marijuana, Marketplac...   
19     Android, Building Maintenance, Collaboration, ...   
20           Cyber Security, Enterprise Software, Mobile   
21                                 Consumer, iOS, Mobile   
22                    Android, Apps, Mobile, Mobile Apps   
23                 Biotechnology, Genetics, Life Science   
24                                        SaaS, Software   
25     Email Marketing, Enterprise Soft

In [64]:
print(e_reduce.shape)
print("The unique startups are {}".format(e_reduce["Organization Name"].nunique()))

e_reduce = e_reduce.drop_duplicates(["Organization Name"], keep = 'first')
result = e_reduce.shape[0] == e_reduce["Organization Name"].nunique()
print("Drop successful? {}".format(result))

(9630, 23)
The unique startups are 9630
Drop successful? True


In [65]:
e_dict = e_reduce.set_index("Organization Name")[['Categories', 'Category Groups',
       'Founded Date', 'Funding Status', 'Operating Status',
       'Total Funding Amount', 'Total Equity Funding Amount']]

e_dict.shape

(9630, 7)

<h2>Merge the funding rounds with startup info</h2>

In [66]:
f_trans_all = f_trans.merge(e_dict, how = 'outer', left_index = True, right_index = True)
f_trans_all.shape

(11926, 16)

In [47]:
f_trans_all

Seed     Series A     Series B     Series C  \
Organization Name                                                          
\t Rhythmm               $120,000         None         None         None   
 CryptoKitties               None  $12,000,000         None         None   
 DOZ                   $1,500,000         None         None         None   
 Harbor                       NaN          NaN          NaN          NaN   
 Justmop.com                 None          NaN         None         None   
 Mahindra Logistics           NaN          NaN          NaN          NaN   
 NimbleRx                    None  $12,000,000  $28,000,000         None   
 Sheerly Genius          $120,000         None         None         None   
 SpotDraft               $550,000         None         None         None   
 Tenfold               $1,350,000  $15,800,000         None         None   
 onX                         None  $20,300,000         None         None   
!hey software                None  $10,000,000         None         None   
/dev/color                    NaN          NaN          NaN          NaN   
10 By 10                 $120,000         None         None         None   
10% Happier            $1,400,000         None         None         None   
1000 Markets             $500,000         None         None         None   
1000memories              $15,000   $2,520,000         None         None   
1006.tv                      None         None  $10,000,000         None   
100Plus                  $750,000         None         None         None   
101 Edu                       NaN         None         None         None   
1010data                      NaN          NaN          NaN          NaN   
10X Genomics                 None         None  $55,000,000  $55,000,000   
11 Honore                    None   $8,000,000         None         None   
117go                        None          NaN         None         None   
123Compliance                 NaN          NaN          NaN          NaN   
123Feng.Com                  None  $13,000,000         None         None   
140 Proof                     NaN          NaN          NaN          NaN   
15Five                 $2,200,000         None         None         None   
17hats                 $1,250,000   $4,000,000         None         None   
17zuoye                       NaN          NaN          NaN          NaN   
...                           ...          ...          ...          ...   
true[x]                      None   $4,200,000   $8,000,000  $11,650,000   
turntable.fm                 None   $7,000,000         None         None   
tuul                   $2,400,000         None         None         None   
twoXAR                 $3,400,000  $10,000,000         None         None   
uBeam                    $750,000  $10,000,000  $20,000,000         None   
uBiome                       None   $4,500,000  $22,000,000         None   
uCiC ( you see I See)    $500,000         None         None         None   
uVore                         NaN         None         None         None   
unGlue                 $1,500,000         None         None         None   
unspun, inc.                  NaN         None         None         None   
userfox                       NaN          NaN          NaN          NaN   
vArmour                      None         None  $15,000,000         None   
vAuto                         NaN          NaN          NaN          NaN   
vLinx                        None         None  $10,000,000         None   
vidIQ                  $2,000,000         None         None         None   
videogram                $500,000   $4,099,999         None         None   
vip.com                      None   $9,190,000  $41,240,000         None   
vvall                         NaN         None         None         None   
webtide                       NaN          NaN          NaN          NaN   
wedgies                  $700,000         None         None         None   
w

In [67]:
print(f_trans_all.dropna(axis = 0, how = "any", thresh = 11).shape)
f_trans_all.dropna(axis = 0, how = "any", thresh = 11)

(2537, 16)


Seed     Series A     Series B     Series C  \
Organization Name                                                         
 NimbleRx                   None  $12,000,000  $28,000,000         None   
 Tenfold              $1,350,000  $15,800,000         None         None   
1000memories             $15,000   $2,520,000         None         None   
10X Genomics                None         None  $55,000,000  $55,000,000   
17hats                $1,250,000   $4,000,000         None         None   
1mg                         None   $6,000,000  $16,000,000  $15,000,000   
23andMe                     None   $8,953,320  $13,600,000  $22,220,289   
2U Inc.                     None  $10,000,000  $20,000,000  $32,500,000   
2nd Address           $2,300,000  $10,000,000         None         None   
3-V Biosciences             None         None  $30,000,000  $20,000,000   
33Across                    None         None   $9,000,000  $13,100,000   
3D Robotics                 None   $5,075,000  $30,000,000  $50,000,000   
3Scan                       None   $6,673,012  $14,000,000         None   
3fbio                   £461,000   £6,160,000         None         None   
41st Parameter              None   $4,350,000  $11,200,000  $10,070,000   
42Floors                $400,000   $5,000,000  $12,000,000         None   
4moms                        NaN  $20,000,000  $40,000,000         None   
500Friends            $1,400,000   $4,500,000   $5,000,000         None   
500px                       None          NaN  $13,000,000         None   
51.com                      None   $6,000,000  $15,000,000         None   
5min Media                  None   $5,000,000   $7,500,000         None   
6 River Systems             None  $15,000,000  $25,000,000         None   
6sense                      None  $16,000,000  $20,000,000         None   
99.co                 $1,600,000   $7,900,000         None         None   
99designs                   None  $35,000,000  $10,000,000         None   
ABILITY Network             None         None   $2,450,000  $27,000,000   
ACV Auctions                None         None  $15,000,000  $31,000,000   
AGEIA Technologies          None         None          NaN  $27,500,000   
AVEO Pharmaceuticals        None  $15,000,000  $42,700,000         None   
AXIOS Media                 None  $10,000,000  $20,000,000         None   
...                          ...          ...          ...          ...   
mParticle             $3,000,000         None  $17,500,000  $35,000,000   
mValent                     None         None   $2,208,000   $6,000,000   
madKast                  $15,000     $300,000         None         None   
migme                        NaN  $10,000,000  $13,500,000   $8,900,000   
moka5                       None         None  $15,000,000  $21,000,000   
nLIGHT Corp.                None         None  $43,600,000  $13,200,000   
netvmg                      None  $10,000,000  $57,000,000  $11,190,000   
ngmoco                      None         None  $10,000,000  $24,200,000   
npm                   $2,600,000   $8,000,000         None         None   
obopay                      None  $10,000,000   $7,000,000  $29,000,000   
oneforty                     NaN   $1,980,000         None         None   
ooma                        None   $8,000,000  $18,000,000  $16,000,000   
otonomo                     None   $3,000,000  $25,000,000         None   
payleven                    None   €2,000,000  €12,500,000         None   
picoChip                    None   $7,000,000  $17,000,000  $20,500,000   
rPath                       None   $6,400,000   $9,100,000  $10,000,000   
scanR                       None   $4,650,000   $8,000,000         None   
sovrn                       None         None   $3,300,000   $7,100,000   
storefront            $1,600,000   $7,300,000         None         None   
sulekha.com                 None  $10,000,000   $6,500,000  $28,100,000   
textPlus                    None         None   $8,15

## Convert Dates


In [68]:
# x = f_trans_all.loc["99.co", "A_date_dt"] - f_trans_all.loc["99.co", "Seed_date_dt"]
f_trans_all["Seed_date_dt"] = pd.to_datetime(f_trans_all["Seed_date"], errors="ignore", format = "%b %d, %Y")
f_trans_all["A_date_dt"] = pd.to_datetime(f_trans_all["A_date"], errors="ignore", format = "%b %d, %Y")
f_trans_all["Seed_A"] = f_trans_all["A_date_dt"] - f_trans_all["Seed_date_dt"]
f_trans_all.dropna(subset=["Seed_date_dt", "A_date_dt"])[["Seed_date_dt", "A_date_dt", "Seed_A"]]

Seed_date_dt  A_date_dt     Seed_A
Organization Name                                       
 Tenfold                2015-07-30 2016-12-27   516 days
1000memories            2010-01-01 2011-02-16   411 days
17hats                  2015-02-18 2015-10-15   239 days
2nd Address             2015-06-11 2015-08-06    56 days
3fbio                   2017-07-16 2018-04-04   262 days
42Floors                2011-12-01 2012-11-16   351 days
4moms                   2006-01-01 2012-08-09  2412 days
500Friends              2009-12-15 2012-03-14   820 days
99.co                   2015-01-29 2017-04-06   798 days
ADstruc                 2010-09-28 2012-06-01   612 days
Acacia Communications   2009-01-01 2009-05-15   134 days
Accion Systems          2015-01-05 2016-05-12   493 days
ActionIQ                2014-08-21 2017-04-03   956 days
AdRoll                  2006-03-01 2008-09-16   930 days
Adcade                  2012-11-01 2013-11-01   365 days
Adelphic                2011-07-21 2012-12-04   502 days
Adore Me                2012-11-06 2012-04-01  -219 days
AirHelp                 2014-03-14 2016-08-03   873 days
Airbnb                  2009-04-01 2010-11-10   588 days
Airspace Systems Inc.   2016-11-18 2018-03-12   479 days
Airtable                2015-02-25 2015-06-29   124 days
Airware                 2013-03-31 2013-05-15    45 days
Algolia                 2014-06-19 2015-05-20   335 days
Algorithmia             2014-08-18 2017-06-23  1040 days
Allbirds                2016-03-01 2016-09-07   190 days
Allset                  2016-09-20 2017-10-17   392 days
Alooma                  2014-08-18 2016-03-14   574 days
AltSchool               2013-08-01 2014-03-18   229 days
Althea Inc.             2015-10-15 2016-05-26   224 days
Amino Apps              2014-07-09 2015-09-24   442 days
...                            ...        ...        ...
ZappRx                  2014-01-30 2015-04-02   427 days
Zarget                  2016-05-09 2016-11-02   177 days
Zencoder                2010-02-01 2011-04-12   435 days
Zenefits                2013-07-26 2014-01-22   180 days
Zenreach                2013-07-01 2015-03-01   608 days
Zero                    2016-09-20 2017-10-23   398 days
Zilingo                 2015-11-09 2016-09-05   301 days
ZipMatch                2014-02-28 2015-03-24   389 days
Zipline                 2011-11-01 2012-10-16   350 days
Zoomdata                2012-11-13 2013-07-10   239 days
Zoopla                  2008-01-15 2009-01-19   370 days
Zoox                    2015-07-03 2016-10-04   459 days
Zugata                  2015-10-13 2016-11-14   398 days
Zumper                  2012-09-27 2014-03-04   523 days
Zylo                    2016-10-13 2018-01-23   467 days
b8ta                    2015-12-02 2016-09-28   301 days
dotCloud                2010-08-01 2011-03-30   241 days
gdgt                    2009-07-01 2010-04-07   280 days
iConclude               2005-02-01 2005-08-31   211 days
isocket                 2011-09-28 2012-10-31   399 days
just.me Inc             2011-07-30 2011-12-31   154 days
madKast                 2007-08-01 2007-10-05    65 days
migme                   2016-09-30 2007-05-07 -3434 days
npm                     2014-02-11 2015-04-01   414 days
oneforty                2009-09-10 2010-01-11   123 days
storefront              2013-06-26 2014-04-14   292 days
tray.io                 2015-04-16 2016-11-17   581 days
twoXAR                  2015-11-19 2018-03-19   851 days
uBeam                   2012-07-10 2014-10-30   842 days
videogram               2013-04-24 2015-02-09   656 days

[795 rows x 3 columns]

In [69]:
dates_string = ["Seed_date", "A_date", "B_date", "C_date"]
dates = []
def convert_datetime(dates_string, df):
    for date in dates_string:
        new_col_name = date + "_dt"
        dates.append(new_col_name)
        df[new_col_name] = pd.to_datetime(df[date], errors="ignore", format = "%b %d, %Y")
        
def time_to_next_round(dates, df):
    for i in range(len(dates)-1):
        new_col_name = dates[i][0] + "_to_" + dates[i+1][0]
        
        df[new_col_name] = df[dates[i+1]] - df[dates[i]]

In [70]:
convert_datetime(dates_string, f_trans_all)
time_to_next_round(dates, f_trans_all)

In [71]:
f_trans_all.dropna(subset = [["S_to_A", "A_to_B", "B_to_C"]], thresh = 2)[["index", "Seed_date_dt", "A_date_dt", "B_date_dt", "C_date_dt",
            "S_to_A", "A_to_B", "B_to_C"]]

index Seed_date_dt  A_date_dt  \
Organization Name                                                      
1mg                                      1mg          NaT 2015-04-20   
23andMe                              23andMe          NaT 2007-10-03   
2U Inc.                              2U Inc.          NaT 2009-06-21   
3D Robotics                      3D Robotics          NaT 2012-11-05   
41st Parameter                41st Parameter          NaT 2005-05-16   
42Floors                            42Floors   2011-12-01 2012-11-16   
4moms                                  4moms   2006-01-01 2012-08-09   
500Friends                        500Friends   2009-12-15 2012-03-14   
Acacia Communications  Acacia Communications   2009-01-01 2009-05-15   
AccelChip                          AccelChip          NaT 2001-01-01   
Achaogen                            Achaogen          NaT 2004-08-01   
Acopia Networks              Acopia Networks          NaT 2002-10-03   
ActionIQ                            ActionIQ   2014-08-21 2017-04-03   
AdReady                              AdReady          NaT 2007-07-01   
AdRoll                                AdRoll   2006-03-01 2008-09-16   
Adap.tv                              Adap.tv          NaT 2007-08-08   
AdaptiveBlue                    AdaptiveBlue          NaT 2007-02-20   
Adelphic                            Adelphic   2011-07-21 2012-12-04   
Admeld                                Admeld          NaT 2008-11-10   
Adnexus                              Adnexus          NaT 2003-03-01   
Adometry By Google        Adometry By Google          NaT 2007-02-07   
Aeluros                              Aeluros          NaT 2001-08-01   
Affirmed Networks          Affirmed Networks          NaT 2010-06-18   
Airbnb                                Airbnb   2009-04-01 2010-11-10   
Airtable                            Airtable   2015-02-25 2015-06-29   
Airware                              Airware   2013-03-31 2013-05-15   
Akorri Networks              Akorri Networks          NaT 2005-05-05   
Aktino                                Aktino          NaT 2003-05-15   
Alcresta                            Alcresta          NaT 2012-04-17   
Algolia                              Algolia   2014-06-19 2015-05-20   
...                                      ...          ...        ...   
Zenefits                            Zenefits   2013-07-26 2014-01-22   
Zenreach                            Zenreach   2013-07-01 2015-03-01   
Zenverge                            Zenverge          NaT 2007-02-05   
Zerto                                  Zerto          NaT 2010-05-25   
ZestFinance                      ZestFinance          NaT 2011-07-21   
Zilingo                              Zilingo   2015-11-09 2016-09-05   
Zinka Logistics              Zinka Logistics          NaT 2015-07-22   
Zipline                              Zipline   2011-11-01 2012-10-16   
Zocdoc                                Zocdoc          NaT 2008-08-18   
ZoomCar                              ZoomCar          NaT 2014-10-31   
Zoomdata                            Zoomdata   2012-11-13 2013-07-10   
Zoopla                                Zoopla   2008-01-15 2009-01-19   
Zuoyebang                          Zuoyebang          NaT 2015-09-03   
dataxu                                dataxu          NaT 2009-04-17   
eero                                    eero          NaT 2015-11-03   
iConclude                          iConclude   2005-02-01 2005-08-31   
iControl Networks          iControl Networks          NaT 2006-04-26   
iSkoot                                iSkoot          NaT 2006-11-29   
iYogi                                  iYogi          NaT 2007-01-01   
m-Qube                                m-Qube          NaT 2002-09-18   
migme                                  migme   2016-09-30 2007-05-07   
netvmg                                netvmg          NaT 2000-05-01   
obopay                                obopay          NaT 2006-03-08   
ooma   

In [72]:
f_trans_all.columns

Index(['Seed', 'Series A', 'Series B', 'Series C', 'Seed_date', 'A_date',
       'B_date', 'C_date', 'index', 'Categories', 'Category Groups',
       'Founded Date', 'Funding Status', 'Operating Status',
       'Total Funding Amount', 'Total Equity Funding Amount', 'Seed_date_dt',
       'A_date_dt', 'Seed_A', 'B_date_dt', 'C_date_dt', 'S_to_A', 'A_to_B',
       'B_to_C'],
      dtype='object')

## Convert Currencies

In [73]:
# for money in f_trans_all["Seed"]:
#     print(money)

In [74]:
f_trans.iloc[8:25, :]

Funding Type             Seed     Series A     Series B     Series C  \
Organization Name                                                      
 onX                     None  $20,300,000         None         None   
!hey software            None  $10,000,000         None         None   
10 By 10             $120,000         None         None         None   
10% Happier        $1,400,000         None         None         None   
1000 Markets         $500,000         None         None         None   
1000memories          $15,000   $2,520,000         None         None   
1006.tv                  None         None  $10,000,000         None   
100Plus              $750,000         None         None         None   
101 Edu                   NaN         None         None         None   
10X Genomics             None         None  $55,000,000  $55,000,000   
11 Honore                None   $8,000,000         None         None   
117go                    None          NaN         None         None   
123Feng.Com              None  $13,000,000         None         None   
15Five             $2,200,000         None         None         None   
17hats             $1,250,000   $4,000,000         None         None   
19pay                    None         None   $6,230,000         None   
1DocWay                   NaN         None         None         None   

Funding Type          Seed_date        A_date        B_date        C_date  \
Organization Name                                                           
 onX                       None  Feb 16, 2018          None          None   
!hey software              None  Feb 23, 2000          None          None   
10 By 10            Jul 1, 2017          None          None          None   
10% Happier        Jan 21, 2014          None          None          None   
1000 Markets       May 15, 2009          None          None          None   
1000memories        Jan 1, 2010  Feb 16, 2011          None          None   
1006.tv                    None          None  Jul 31, 2014          None   
100Plus            Nov 30, 2011          None          None          None   
101 Edu            Mar 15, 2016          None          None          None   
10X Genomics               None          None  Jan 12, 2015  Mar 17, 2016   
11 Honore                  None  Mar 19, 2018          None          None   
117go                      None   Jul 1, 2012          None          None   
123Feng.Com                None  Oct 27, 2015          None          None   
15Five              Oct 2, 2014          None          None          None   
17hats             Feb 18, 2015  Oct 15, 2015          None          None   
19pay                      None          None   Jan 1, 2010          None   
1DocWay            Aug 10, 2011          None          None          None   

Funding Type               index  
Organization Name                 
 onX                         onX  
!hey software      !hey software  
10 By 10                10 By 10  
10% Happier          10% Happier  
1000 Markets        1000 Markets  
1000memories        1000memories  
1006.tv                  1006.tv  
100Plus                  100Plus  
101 Edu                  101 Edu  
10X Genomics        10X Genomics  
11 Honore              11 Honore  
117go                      117go  
123Feng.Com          123Feng.Com  
15Five                    15Five  
17hats                    17hats  
19pay                      19pay  
1DocWay                  1DocWay

In [75]:
amount_list = ["Seed", "Series A", "Series B", "Series C"]

def fill_na(amount_list, df):
    for fund in amount_list:
        df[fund] = df[fund].fillna("$0")

In [76]:
fill_na(amount_list, f_trans_all)
f_trans_all

Seed     Series A     Series B     Series C  \
Organization Name                                                          
\t Rhythmm               $120,000           $0           $0           $0   
 CryptoKitties                 $0  $12,000,000           $0           $0   
 DOZ                   $1,500,000           $0           $0           $0   
 Harbor                        $0           $0           $0           $0   
 Justmop.com                   $0           $0           $0           $0   
 Mahindra Logistics            $0           $0           $0           $0   
 NimbleRx                      $0  $12,000,000  $28,000,000           $0   
 Sheerly Genius          $120,000           $0           $0           $0   
 SpotDraft               $550,000           $0           $0           $0   
 Tenfold               $1,350,000  $15,800,000           $0           $0   
 onX                           $0  $20,300,000           $0           $0   
!hey software                  $0  $10,000,000           $0           $0   
/dev/color                     $0           $0           $0           $0   
10 By 10                 $120,000           $0           $0           $0   
10% Happier            $1,400,000           $0           $0           $0   
1000 Markets             $500,000           $0           $0           $0   
1000memories              $15,000   $2,520,000           $0           $0   
1006.tv                        $0           $0  $10,000,000           $0   
100Plus                  $750,000           $0           $0           $0   
101 Edu                        $0           $0           $0           $0   
1010data                       $0           $0           $0           $0   
10X Genomics                   $0           $0  $55,000,000  $55,000,000   
11 Honore                      $0   $8,000,000           $0           $0   
117go                          $0           $0           $0           $0   
123Compliance                  $0           $0           $0           $0   
123Feng.Com                    $0  $13,000,000           $0           $0   
140 Proof                      $0           $0           $0           $0   
15Five                 $2,200,000           $0           $0           $0   
17hats                 $1,250,000   $4,000,000           $0           $0   
17zuoye                        $0           $0           $0           $0   
...                           ...          ...          ...          ...   
true[x]                        $0   $4,200,000   $8,000,000  $11,650,000   
turntable.fm                   $0   $7,000,000           $0           $0   
tuul                   $2,400,000           $0           $0           $0   
twoXAR                 $3,400,000  $10,000,000           $0           $0   
uBeam                    $750,000  $10,000,000  $20,000,000           $0   
uBiome                         $0   $4,500,000  $22,000,000           $0   
uCiC ( you see I See)    $500,000           $0           $0           $0   
uVore                          $0           $0           $0           $0   
unGlue                 $1,500,000           $0           $0           $0   
unspun, inc.                   $0           $0           $0           $0   
userfox                        $0           $0           $0           $0   
vArmour                        $0           $0  $15,000,000           $0   
vAuto                          $0           $0           $0           $0   
vLinx                          $0           $0  $10,000,000           $0   
vidIQ                  $2,000,000           $0           $0           $0   
videogram                $500,000   $4,099,999           $0           $0   
vip.com                        $0   $9,190,000  $41,240,000           $0   
vvall                          $0           $0           $0           $0   
webtide                        $0           $0           $0           $0   
wedgies                  $700,000           $0           $0           $0   
w

In [77]:
funds = ["Seed", "Series A", "Series B", "Series C"]
currency_col = ["S_c", "A_c", "B_c", "C_c"]


def fund_currency(funds, df):
    for fund in funds:
        new_col_name = ""
        if fund == "Seed":
            new_col_name = fund[0] + "_c"
        else:
            new_col_name = fund[-1] + "_c"
        df[new_col_name] = df[fund].str.replace("[0-9]|[,]", "")



def fund_to_int(funds, df):
    for fund in funds:
        new_col_name = ""
        if fund == "Seed":
            new_col_name = fund[0] + "_int"
        else:
            new_col_name = fund[-1] + "_int"
        df[new_col_name] = df[fund].str.replace("[^0-9]+|[,]", "")

        
def currencies(currency_col, df):
    list_of_currencies = []
    
    for currencies in currency_col:
        for currency in df[currencies].unique():
            list_of_currencies.append(currency)
    return list_of_currencies

In [78]:
fund_currency(funds, f_trans_all)
fund_to_int(funds, f_trans_all)

In [79]:
curr = currencies(currency_col, f_trans_all)
curr

['$',
 '£',
 '₩',
 'SGD',
 '€',
 'NZ$',
 'A$',
 'CA$',
 'MYR',
 'CN¥',
 '₹',
 '¥',
 '$',
 '£',
 'CA$',
 'CN¥',
 '€',
 '₹',
 'A$',
 '¥',
 'SGD',
 'CHF',
 '$',
 'CN¥',
 '£',
 '€',
 '₹',
 'CA$',
 '¥',
 'NOK',
 '$',
 'CN¥',
 '€',
 '₹',
 'R$',
 'A$',
 '£']

In [80]:
f_trans_all.columns

Index(['Seed', 'Series A', 'Series B', 'Series C', 'Seed_date', 'A_date',
       'B_date', 'C_date', 'index', 'Categories', 'Category Groups',
       'Founded Date', 'Funding Status', 'Operating Status',
       'Total Funding Amount', 'Total Equity Funding Amount', 'Seed_date_dt',
       'A_date_dt', 'Seed_A', 'B_date_dt', 'C_date_dt', 'S_to_A', 'A_to_B',
       'B_to_C', 'S_c', 'A_c', 'B_c', 'C_c', 'S_int', 'A_int', 'B_int',
       'C_int'],
      dtype='object')

In [81]:
f_trans_all.head(10).iloc[:,16:]

Seed_date_dt  A_date_dt   Seed_A  B_date_dt C_date_dt  \
Organization Name                                                            
\t Rhythmm             2018-01-04        NaT      NaT        NaT       NaT   
 CryptoKitties                NaT 2018-03-21      NaT        NaT       NaT   
 DOZ                   2014-09-15        NaT      NaT        NaT       NaT   
 Harbor                       NaT        NaT      NaT        NaT       NaT   
 Justmop.com                  NaT 2017-11-21      NaT        NaT       NaT   
 Mahindra Logistics           NaT        NaT      NaT        NaT       NaT   
 NimbleRx                     NaT 2015-10-13      NaT 2017-10-24       NaT   
 Sheerly Genius        2018-01-04        NaT      NaT        NaT       NaT   
 SpotDraft             2017-10-25        NaT      NaT        NaT       NaT   
 Tenfold               2015-07-30 2016-12-27 516 days        NaT       NaT   

                       S_to_A   A_to_B  B_to_C S_c A_c B_c C_c    S_int  \
Organization Name                                                         
\t Rhythmm                NaT      NaT     NaT   $   $   $   $   120000   
 CryptoKitties            NaT      NaT     NaT   $   $   $   $        0   
 DOZ                      NaT      NaT     NaT   $   $   $   $  1500000   
 Harbor                   NaT      NaT     NaT   $   $   $   $        0   
 Justmop.com              NaT      NaT     NaT   $   $   $   $        0   
 Mahindra Logistics       NaT      NaT     NaT   $   $   $   $        0   
 NimbleRx                 NaT 742 days     NaT   $   $   $   $        0   
 Sheerly Genius           NaT      NaT     NaT   $   $   $   $   120000   
 SpotDraft                NaT      NaT     NaT   $   $   $   $   550000   
 Tenfold             516 days      NaT     NaT   $   $   $   $  1350000   

                         A_int     B_int C_int  
Organization Name                               
\t Rhythmm                   0         0     0  
 CryptoKitties        12000000         0     0  
 DOZ                         0         0     0  
 Harbor                      0         0     0  
 Justmop.com                 0         0     0  
 Mahindra Logistics          0         0     0  
 NimbleRx             12000000  28000000     0  
 Sheerly Genius              0         0     0  
 SpotDraft                   0         0     0  
 Tenfold              15800000         0     0

In [82]:
f_trans_all.drop_duplicates(subset = ["Seed_c", "A_c", "B_c", "C_c"], )[["Seed_c", "A_c", "B_c", "C_c"]]

KeyError: 'Seed_c'

### Output funds to csv

In [92]:
f_trans_all

Seed     Series A     Series B     Series C  \
Organization Name                                                          
\t Rhythmm               $120,000           $0           $0           $0   
 CryptoKitties                 $0  $12,000,000           $0           $0   
 DOZ                   $1,500,000           $0           $0           $0   
 Harbor                        $0           $0           $0           $0   
 Justmop.com                   $0           $0           $0           $0   
 Mahindra Logistics            $0           $0           $0           $0   
 NimbleRx                      $0  $12,000,000  $28,000,000           $0   
 Sheerly Genius          $120,000           $0           $0           $0   
 SpotDraft               $550,000           $0           $0           $0   
 Tenfold               $1,350,000  $15,800,000           $0           $0   
 onX                           $0  $20,300,000           $0           $0   
!hey software                  $0  $10,000,000           $0           $0   
/dev/color                     $0           $0           $0           $0   
10 By 10                 $120,000           $0           $0           $0   
10% Happier            $1,400,000           $0           $0           $0   
1000 Markets             $500,000           $0           $0           $0   
1000memories              $15,000   $2,520,000           $0           $0   
1006.tv                        $0           $0  $10,000,000           $0   
100Plus                  $750,000           $0           $0           $0   
101 Edu                        $0           $0           $0           $0   
1010data                       $0           $0           $0           $0   
10X Genomics                   $0           $0  $55,000,000  $55,000,000   
11 Honore                      $0   $8,000,000           $0           $0   
117go                          $0           $0           $0           $0   
123Compliance                  $0           $0           $0           $0   
123Feng.Com                    $0  $13,000,000           $0           $0   
140 Proof                      $0           $0           $0           $0   
15Five                 $2,200,000           $0           $0           $0   
17hats                 $1,250,000   $4,000,000           $0           $0   
17zuoye                        $0           $0           $0           $0   
...                           ...          ...          ...          ...   
true[x]                        $0   $4,200,000   $8,000,000  $11,650,000   
turntable.fm                   $0   $7,000,000           $0           $0   
tuul                   $2,400,000           $0           $0           $0   
twoXAR                 $3,400,000  $10,000,000           $0           $0   
uBeam                    $750,000  $10,000,000  $20,000,000           $0   
uBiome                         $0   $4,500,000  $22,000,000           $0   
uCiC ( you see I See)    $500,000           $0           $0           $0   
uVore                          $0           $0           $0           $0   
unGlue                 $1,500,000           $0           $0           $0   
unspun, inc.                   $0           $0           $0           $0   
userfox                        $0           $0           $0           $0   
vArmour                        $0           $0  $15,000,000           $0   
vAuto                          $0           $0           $0           $0   
vLinx                          $0           $0  $10,000,000           $0   
vidIQ                  $2,000,000           $0           $0           $0   
videogram                $500,000   $4,099,999           $0           $0   
vip.com                        $0   $9,190,000  $41,240,000           $0   
vvall                          $0           $0           $0           $0   
webtide                        $0           $0           $0           $0   
wedgies                  $700,000           $0           $0           $0   
w

In [93]:
f_trans_all.to_csv("../output/wrangle_full.csv")

In [83]:
list(set(curr))

['SGD',
 'NZ$',
 '€',
 'A$',
 'R$',
 '£',
 '₹',
 'CHF',
 '¥',
 'MYR',
 '₩',
 'CN¥',
 'NOK',
 'CA$',
 '$']

# rough code here

In [84]:
# f_trans_all["Seed_int"] = f_trans_all["Seed"].str.replace("[^0-9]|[,]", "")
# f_trans_all["A_int"] = f_trans_all["Series A"].str.replace("[^0-9]+|[,]", "")
# f_trans_all["B_int"] = f_trans_all["Series B"].str.replace("[^0-9]+|[,]", "")
# f_trans_all["C_int"] = f_trans_all["Series C"].str.replace("[^0-9]+|[,]", "")
# f_trans_all

In [85]:
# f_trans_all["Seed_c"] = f_trans_all["Seed"].str.replace("[0-9]+|[,]", "")
# f_trans_all["A_c"] = f_trans_all["Series A"].str.replace("[0-9]+|[,]", "")
# f_trans_all["B_c"] = f_trans_all["Series B"].str.replace("[0-9]+|[,]", "")
# f_trans_all["C_c"] = f_trans_all["Series C"].str.replace("[0-9]+|[,]", "")
# f_trans_all[f_trans_all.Seed_c.str.contains('£|₩|¥|A$|CA$|R$')][["Seed_c", "A_c", "B_c", "C_c"]]

In [86]:
currency = list(f_trans_all["Seed"].str.replace("[0-9]+|[,]", "").unique())
currency

['$', '£', '₩', 'SGD', '€', 'NZ$', 'A$', 'CA$', 'MYR', 'CN¥', '₹', '¥']

In [87]:
currency.append(list(f_trans_all["Series C"].str.replace("[0-9]+|[,]", "").unique()))

currency

['$',
 '£',
 '₩',
 'SGD',
 '€',
 'NZ$',
 'A$',
 'CA$',
 'MYR',
 'CN¥',
 '₹',
 '¥',
 ['$', 'CN¥', '€', '₹', 'R$', 'A$', '£']]

In [88]:
f_trans_all["Seed_currency"] = f_trans_all["Seed"].str.replace("[0-9]+|[,]", "")

In [89]:
f_trans_all["Seed_currency"].unique()

array(['$', '£', '₩', 'SGD', '€', 'NZ$', 'A$', 'CA$', 'MYR', 'CN¥', '₹',
       '¥'], dtype=object)

In [90]:
currency = re.compile(r'^[A-z]?[$£₩¥€]')

In [207]:
# currency_list = []

# for money in f_trans_all["Seed"]:
#     amount = re.compile(r'^[A-z]?[$£₩¥€]')
# #     if money :
# #         print(money)
#     currency = amount.search(money)
#     if currency is not None:
#         currency_list.append(currency.group())
    
# f_trans_all["Seed_currency"] = currency_list

ValueError: Length of values does not match length of index

In [196]:
%time list(set([1, 1.5, 5, 0.8, 100, 50, 0.78]))

CPU times: user 9 µs, sys: 1 µs, total: 10 µs
Wall time: 14.1 µs


[0.8, 1, 1.5, 0.78, 100, 5, 50]

In [192]:
rate = set([1, 1.5, 5, 0.8, 100, 50, 0.78])

In [193]:
curr = set(currency_list)

In [194]:
curr
rate

{0.78, 0.8, 1, 1.5, 5, 50, 100}

In [138]:
f_trans_all.dtypes

Seed                                    object
Series A                                object
Series B                                object
Series C                                object
Seed_date                               object
A_date                                  object
B_date                                  object
C_date                                  object
index                                   object
Categories                              object
Category Groups                         object
Founded Date                            object
Funding Status                          object
Operating Status                        object
Total Funding Amount                    object
Total Equity Funding Amount             object
Seed_date_dt                    datetime64[ns]
A_date_dt                       datetime64[ns]
Seed_A                         timedelta64[ns]
B_date_dt                       datetime64[ns]
C_date_dt                       datetime64[ns]
S_to_A       

In [132]:
f_trans_all["Seed_int"] = f_trans_all["Seed"].str.replace("^[A-z]?[$£₩¥€]", "")
f_trans_all["Seed_int"] = f_trans_all["Seed_int"].str.replace(",", "")

In [133]:
f_trans_all[["Seed_int", "Seed"]]

Seed_int        Seed
Organization Name                                          
 Justmop.com                               None        None
 NimbleRx                                  None        None
 SpotDraft                               550000    $550,000
/dev/color                               120000    $120,000
10 By 10                                 120000    $120,000
1000memories                               None        None
1006.tv                                    None        None
140 Proof                                  None        None
1Huddle                                  125000    $125,000
1mg                                        None        None
1stdibs                                     NaN         NaN
20n                                      120000    $120,000
20x200                                     None        None
2345.com                                   None        None
23andMe                                    None        None
247tickets                                  NaN         NaN
280 North                                   NaN         NaN
2Wire                                      None        None
3-V Biosciences                            None        None
37coins                                  500000    $500,000
3Jam                                       None        None
3LM                                     1500000  $1,500,000
3TEN8.AI                                1670000  $1,670,000
3VR                                        None        None
3ware                                       NaN         NaN
41st Parameter                             None        None
42 Technologies                             NaN         NaN
42Floors                                 400000    $400,000
43Layers                                    NaN         NaN
4INFO                                      None        None
...                                         ...         ...
payleven                                   None        None
play140                                     NaN         NaN
quixi                                      None        None
rakam                                    150000    $150,000
reMail                                      NaN         NaN
resin.io                                   None        None
seeUthere.com                               NaN         NaN
simplifund                               125000    $125,000
slinkset                                    NaN         NaN
so-sure                                  150000    $150,000
sourceeasy                               126500    $126,500
sp0n                                       None        None
springcleaning.ae                          None        None
storefront                                 None        None
strapping                                100000    $100,000
tCell                                      None        None
tagMonkey                                125000    $125,000
terraspring                                 NaN         NaN
textPlus                                   None        None
uBiome                                   120000    $120,000
uShip                                      None        None
unspun, inc.                                NaN         NaN
userfox                                     NaN         NaN
vLine                                   1500000  $1,500,000
videogram                                  None        None
vip.com                                    None        None
wedgies                                  550000    $550,000
wifi.com.ng                             2100000  $2,100,000
xendit                                   120000    $120,000
zPREDICTA (formerly Ixchel Scientific)   120000    $120,000

[5006 rows x 2 columns]

In [119]:
f_trans_all[f_trans_all["Seed_int"].str.contains("^[A-z]?[$£₩¥€]") == True]

Empty DataFrame
Columns: [Seed, Series A, Series B, Series C, Seed_date, A_date, B_date, C_date, index, Categories, Category Groups, Founded Date, Funding Status, Operating Status, Total Funding Amount, Total Equity Funding Amount, Seed_date_dt, A_date_dt, Seed_A, B_date_dt, C_date_dt, S_to_A, A_to_B, B_to_C, Seed_int]
Index: []

[0 rows x 25 columns]

In [308]:
f_trans_all.to_csv("../output/f_trans_all.csv")

In [31]:
# pd.to_datetime(f_trans_all, "A_date", format="%b %d, %Y", errors = 'coerce')

TypeError: to_datetime() got multiple values for argument 'errors'

In [32]:
# currency_to_int(f_trans_all, "Seed", "Seed_int")

ValueError: could not convert string to float: '₹54000000'

In [ ]:
# f_trans_all.datetime

In [ ]:
# f_trans_all[["Seed", "Seed_int"]]

In [ ]:
# f_trans_all[f_trans_all["Seed_int"] == "$1000000"]

In [ ]:
# f_trans_all["Series B"].str.replace(",", "")

## Duplicates are predominantly null values.

In [ ]:
print(f_trans_all[f_trans_all.duplicated(keep = False)].shape)
f_trans_all[f_trans_all.duplicated(keep = False)]